In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import ipympl
import ipywidgets as widgets
import tkinter as tk
from IPython.display import display

In [2]:
class Laguerre:
    def __init__(self, t=20, n=10, num_of_points=100, eps=0.1, beta=2, sigma=4):
        self.t = t
        self.n = n
        self.beta = beta
        self.sigma = sigma
        self.num_of_points = num_of_points
        self.eps = eps
        self.nu = 6
        self.gamma = 1
    def __str__(self):
        return "t: {} n: {} number of points: {} eps: {} beta: {} sigma: {}".format(self.t, self.n, self.num_of_points, self.eps, self.beta, self.sigma)
        
    def laguerre(self):
        l_0 = np.sqrt(self.sigma) * (np.exp(-self.beta * self.t / 2))
        l_1 = np.sqrt(self.sigma) * (1 - self.sigma * self.t) * (np.exp(-self.beta * self.t / 2))

        if self.n == 0:
            return l_0
        if self.n == 1:
            return l_1
        if self.n >= 2:
            l_next = (2 * 2 - 1 - self.t * self.sigma) / 2 * l_1 - (2 - 1) / 2 * l_0
            for j in range(3, self.n + 1):
                l_0 = l_1
                l_1 = l_next
                l_next = (2 * j - 1 - self.t * self.sigma) / j * l_1 - (j - 1) / j * l_0
            return l_next
        
    def tabulate_laguerre(self):
        steps = np.linspace(0, self.t, self.num_of_points)
        y_s = []
        for i in steps:
            self.t = i
            y_s.append(self.laguerre())
        return steps, y_s
    
    def experiment(self):
        self.t = 0
        while True:
            self.t += 0.01
            res = []

            for i in range(self.n + 1):
                x = abs(self.laguerre())
                if x < self.eps:
                    res.append(x)
                    if i == self.n:
                        return self.t, res
                else:
                    break
                    
    def integral_with_rectangles(self, f):
        alpha = self.sigma - self.beta
        self.num_of_points = 100
        steps = np.linspace(0, self.t, self.num_of_points)

        help1 = []
        for i in steps:
            self.t = i
            help1.append(f(i) * self.laguerre() * np.exp(-alpha * i))
        res1 = sum(help1) * self.t / self.num_of_points

        self.num_of_points *= 2
        steps = np.linspace(0, self.t, self.num_of_points)

        help2 = []
        for i in steps:
            self.t = i
            help2.append(f(i) * self.laguerre() * np.exp(-alpha * i))
        res2 = sum(help2) * self.t / self.num_of_points

        while abs(res2 - res1) >= self.eps:
            self.num_of_points *= 2
            res1 = res2

            help2 = []
            for i in steps:
                self.t = i
                help2.append(f(i) * self.laguerre() * np.exp(-alpha * i))
            res2 = sum(help2) * self.t / self.num_of_points
        return res2
    
    def integral_with_rectangles_2(self, f, a, b):
        alpha = self.sigma - self.beta
        self.num_of_points = 100
        steps = np.linspace(0, self.t, self.num_of_points)

        help1 = []
        for i in steps:
            self.t = i
            help1.append(f(i, a, b) * self.laguerre() * np.exp(-alpha * i))
        res1 = sum(help1) * self.t / self.num_of_points

        self.num_of_points *= 2
        steps = np.linspace(0, self.t, self.num_of_points)

        help2 = []
        for i in steps:
            self.t = i
            help2.append(f(i, a, b) * self.laguerre() * np.exp(-alpha * i))
        res2 = sum(help2) * self.t / self.num_of_points

        while abs(res2 - res1) >= self.eps:
            self.num_of_points *= 2
            res1 = res2

            help2 = []
            for i in steps:
                self.t = i
                help2.append(f(i, a, b) * self.laguerre() * np.exp(-alpha * i))
            res2 = sum(help2) * self.t / self.num_of_points
        return res2
    
    def laguerre_transformation(self, f):
        to_return = []
        for i in range(self.n + 1):
            self.n = i
            to_return.append(self.integral_with_rectangles(f))
        return to_return
    
    def laguerre_transformation_2(self, f, a, b):
        to_return = []
        for i in range(self.n + 1):
            self.n = i
            to_return.append(self.integral_with_rectangles_2(f, a, b))
        return to_return
    
    def reverse_laguerre_transformation(self, lst):
        to_return = []
        for i in range(len(lst)):
            self.n = i
            to_return.append(lst[i] * self.laguerre())
        return sum(to_return)

In [3]:
def f(t):
    if 0 <= t <= 2 * np.pi:
        return np.sin(t - np.pi / 2) + 1
    elif t > 2 * np.pi:
        return 0

In [4]:
def gaussian_function(t, nu, lamd):
    return 1 / (lamd * np.sqrt(np.pi * 2)) * np.exp(-(((t - nu) ** 2) / (2 * lamd ** 2)))

In [5]:
def for_case_2():
    n_copy, t_copy = lag_test.n, lag_test.t
    lag_test.n = 20
    val, arr = lag_test.experiment()
    lag_test.t = val
    for i in range(0, 21):
        lag_test.n = i
        t_arr, lag_arr = lag_test.tabulate_laguerre()
        plt.plot(t_arr, lag_arr, label = f'L_{i} (t)')
    print(f't = {val}')
    lag_test.n, lag_test.t = n_copy, t_copy
    plt.title('LAGUERRE\'S FUNCTION')
    plt.grid(True)
    plt.xlabel('t')
    plt.ylabel('lag')
    plt.legend()
    plt.show()

In [6]:
def for_case_5(number):
    t_copy, pts_copy = lag_test.t, lag_test.num_of_points
    arr = lag_test.laguerre_transformation(f)
    res = []
    steps = np.linspace(0, 2*np.pi, 1000)
    for i in steps:
        lag_test.t = i
        res.append(lag_test.reverse_laguerre_transformation(arr))
    lag_test.t, lag_test.num_of_points = t_copy, pts_copy
    plt.title('Simple and Reverse transformation')
    plt.plot(steps, res)
    plt.show()

In [7]:
def for_case_6(number, a, b):
    t_copy, pts_copy = lag_test.t, lag_test.num_of_points
    arr = lag_test.laguerre_transformation_2(gaussian_function, a, b)
    res = []
    steps = np.linspace(0, lag_test.t, 1000)
    for i in steps:
        lag_test.t = i
        res.append(lag_test.reverse_laguerre_transformation(arr))
    lag_test.t, lag_test.num_of_points = t_copy, pts_copy
    plt.title('Simple and Reverse transformation Gausse')
    plt.plot(steps, res)
    plt.show()

In [8]:
number_options=["Tabulate laguerre", "Experiment", "Laguerre Transformation", "Reverse Laguerre Transformation", "Simple and Reverse transformation", "Simple and Reverse Gausse Transformation"]

def test_function(t, n, num_of_p, sigma, beta, eps, nu, gamma, lag_test, option):
    lag_test.n = n
    lag_test.t = t
    lag_test.eps = eps
    lag_test.num_of_points = num_of_p
    lag_test.sigma = sigma
    lag_test.beta = beta
    lag_test.nu = nu
    lag_test.gamma = gamma
    plt.close()
    print(lag_test)
    if option == number_options[0]:
        plt.plot(lag_test.tabulate_laguerre()[0], lag_test.tabulate_laguerre()[1])
        plt.title('Tabulate laguerre')
        plt.show()
    elif option == number_options[1]:
        for_case_2()
    elif option == number_options[2]:
        pts_copy = lag_test.num_of_points
        for i in lag_test.laguerre_transformation(f): print(i)
        lag_test.num_of_points = pts_copy
    elif option == number_options[3]:
        n_copy = lag_test.n
        print("\n" + str(lag_test.reverse_laguerre_transformation(np.array([-5, -2, -1, 0, 3, 5, 7]))))
        lag_test.n = n_copy
    elif option == number_options[4]:
        for_case_5(lag_test.n)
    elif option == number_options[5]:
        for_case_6(lag_test.n, nu, gamma)
        
    global option_to_write
    option_to_write = option

In [9]:
def on_button_click(b, lag_test):
    file = open("MyFile.txt", "w")
    t = str(lag_test.t).replace('.', ',')
    epsilon = str(lag_test.eps).replace('.', ',')
    _nu = str(lag_test.nu).replace('.', ',')
    _gam = str(lag_test.gamma).replace('.', ',')
    to_write = f'{t};{lag_test.n};{lag_test.num_of_points};{epsilon};{lag_test.beta};{lag_test.sigma};{_nu};{_gam};{option_to_write}'
    file.write(to_write)

In [10]:
def read_tab_lag(lines):
    arr_x, arr_y = [], []
    
    for i in range(1, int((len(lines) - 3) / 2) + 1):
        to_add = lines[i].replace(',', '.')[:-1]
        arr_x.append(float(to_add))
        
    for i in range(len(lines) - 100, len(lines)):
        to_add = lines[i].replace(',', '.')[:-1]
        arr_y.append(float(to_add))
    
    plt.plot(arr_x, arr_y)
    plt.show() 

In [11]:
def read_exp(lines):
    i = 2
    x = True
    arr_x, arr_y = [], []
    print(lines[1])
    while i < len(lines):
        if lines[i] == '\n' and lines[i - 1] == '\n':
            plt.plot(arr_x, arr_y)
            arr_x, arr_y = [], []
            i += 1
            continue
        if lines[i] == '\n':
            x = not x
            i += 1
            continue
        if x:
            to_add = lines[i].replace(',', '.')[:-1]
            arr_x.append(float(to_add))
        else:
            to_add = lines[i].replace(',', '.')[:-1]
            arr_y.append(float(to_add))
        i += 1
    plt.title('LAGUERRE\'S FUNCTION')
    plt.grid(True)
    plt.xlabel('t')
    plt.ylabel('lag')
    plt.show()

In [12]:
def read_lag_trans(lines):
    for i in range(1, len(lines)):
        print(lines[i].replace(',', '.'), end='')

In [13]:
def read_rev_lag_trans(lines):
    print(lines[1].replace(',', '.'))

In [14]:
def read_simple_and_reverse(lines):
    steps = np.linspace(0, 2*np.pi, 1000)
    res = []
    for i in range(1, len(steps) + 1):
        to_add = lines[i].replace(',', '.')[:-1]
        res.append(float(to_add))

    plt.plot(steps, res)
    plt.show()

In [15]:
def read_simple_and_reverse_gausse(lines):
    arr_x, arr_y = [], []
    
    for i in range(1, int((len(lines) - 3) / 2) + 1):
        to_add = lines[i].replace(',', '.')[:-1]
        arr_x.append(float(to_add))
        
    for i in range(len(lines) - 1000, len(lines)):
        to_add = lines[i].replace(',', '.')[:-1]
        arr_y.append(float(to_add))
    
    plt.plot(arr_x, arr_y)
    plt.show()

In [16]:
functions = [read_tab_lag, read_exp, read_lag_trans, read_rev_lag_trans, read_simple_and_reverse, read_simple_and_reverse_gausse]

In [17]:
def test_function2():  
    options, filename = number_options, 'MyFile.txt'
    with open(filename, 'r') as f:
        lines = f.readlines()

    functions[options.index(lines[0][:-1])](lines)

In [18]:
def on_button_click2(button):
    test_function2()

In [19]:
lag_test = Laguerre()
widgets.interact(test_function, 
                 t=widgets.IntText(value='7'), 
                 n=widgets.IntText(value='5'), 
                 num_of_p=widgets.IntText(value='100'), 
                 sigma=widgets.IntText(value='4'), 
                 beta=widgets.IntText(value='2'), 
                 eps=widgets.FloatText(value='0.001'),
                 nu=widgets.FloatText(value='6'),
                 gamma=widgets.FloatText(value='1'),
                 lag_test=widgets.fixed(lag_test),
                 option=number_options)

button = widgets.Button(description="Записати у файл!")
display(button)
button.on_click(lambda b: on_button_click(b, lag_test))

interactive(children=(IntText(value=7, description='t'), IntText(value=5, description='n'), IntText(value=100,…

Button(description='Записати у файл!', style=ButtonStyle())

In [20]:
button2 = widgets.Button(description="Показати ту саму штуку на C#")
button2.on_click(on_button_click2)
display(button2)

Button(description='Показати ту саму штуку на C#', style=ButtonStyle())